In [1]:
import wmfdata as wmf

In [2]:
query = """
SELECT
    cohort_date,
    SUM(CAST(retention_actions > 0 AS INT)) / COUNT(1) AS second_day_retention
FROM (
    SELECT
        cohort_date,
        cohort.user_id AS user_id,
        COUNT(retention_actions.dt) AS retention_actions
    FROM (
        SELECT
            CAST(DATE_TRUNC('day', TO_TIMESTAMP(dt)) AS DATE) AS cohort_date,
            event.user_id AS user_id,
            TO_TIMESTAMP(MIN(dt)) AS first_read_dt
        FROM event.inukapageview
        WHERE
            event.client_type = 'kaios-app' AND
            (year = 2020 AND month = 9 AND day >= 9 OR
             year = 2020 AND month > 9 OR
             year > 2020) AND
            DATE_TRUNC('day', TO_TIMESTAMP(dt)) < CURRENT_DATE - INTERVAL '2' DAY
        GROUP BY
            event.user_id,
            DATE_TRUNC('day', TO_TIMESTAMP(dt))
    ) cohort
    LEFT JOIN event.inukapageview retention_actions
    ON
        cohort.user_id = retention_actions.event.user_id AND
        TO_TIMESTAMP(retention_actions.dt) BETWEEN
            first_read_dt + INTERVAL '1' DAY AND
            first_read_dt + INTERVAL '2' DAY
    GROUP BY
        cohort.cohort_date,
        cohort.user_id
) retention_actions_per_user
GROUP BY cohort_date
ORDER BY cohort_date
"""

wmf.spark.run(query, session_type="large")

,cohort_date,second_day_retention
0,2020-09-09,0.136215
1,2020-09-10,0.137292
2,2020-09-11,0.136838
3,2020-09-12,0.137823
4,2020-09-13,0.131592
5,2020-09-14,0.130276
6,2020-09-15,0.127931
7,2020-09-16,0.127399
8,2020-09-17,0.128289
9,2020-09-18,0.128372


In [5]:
_2.style.format({"second_day_retention": wmf.utils.pct_str})

,cohort_date,second_day_retention
0,2020-09-09,13.6%
1,2020-09-10,13.7%
2,2020-09-11,13.7%
3,2020-09-12,13.8%
4,2020-09-13,13.2%
5,2020-09-14,13.0%
6,2020-09-15,12.8%
7,2020-09-16,12.7%
8,2020-09-17,12.8%
9,2020-09-18,12.8%
